In [17]:
# Import libraries and set desired options
import pickle
from pathlib2 import Path
import numpy as np
import pandas as pd
from datetime import datetime
import os

from scipy.sparse import csr_matrix, hstack, vstack, coo_matrix
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

import seaborn as sns

from hyperopt import hp, tpe, fmin, STATUS_OK

In [62]:
# This fixes the broken bson dependency that hyperopt can have
from hyperopt import base
base.have_bson = False

# General notes
In this notebook, we build a basic TF-IDF model to identify Alice and then do it again with feature engineering and data cleaning. Finally, we perform hyperparameter optimization to get the final model.
The results are reflected in the article mentioned in the repository readme.

# Utility functions
We'll need some functions to explore the data. Lets put them here to keep the code organized.

In [18]:
# Code by yorko - lead organizer of MLCourse
# Function for writing predictions to a submission file
# Source: https://www.kaggle.com/kashnitsky/correct-time-aware-cross-validation-scheme
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    """
    Args:
    Predicted labels: array of Alice probabilities
    out_file: path to the output file
    
    Out:
    None: saves results to out_file, returns nothing
    
    
    Function writes the results to a csv file that is ready to be submitted
    into the competition.
    Docstring by Alex Kirko.
    """
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [19]:
# Prep for TF_IDF

def form_sentences(in_list):
    """
    Makes rows for the CountVectorizer to process.
    Is made to use with apply. Works by casting floats as strings
    ignoring NaNs, and joining the strings with whitespace.
    
    Args:
    in_list (DataFrame row): row of identifiers/site names to join
    """
    
    str_list = [str(x) for x in in_list if ~pd.isnull(x)]
    return ' '.join(str_list)

In [20]:
def tokenize_basic(string):
    """
    Performs basic tokenization: splits only by whitespace.
    
    Args:
    string (str): string to be tokenized
    Out:
    (list): list of tokens
    """
    return string.split(' ')

In [21]:
def tokenize_domain_name(string):    
    """
    Extracts domain names from a website list string.
    
    Args:
    string (str): string to be tokenized
    Out:
    domains (list): list of tokens
    """
    sites = string.split(' ')
    domains = []
    for site in sites:
        try:
            domain_name = site.split('.')[-2]
        except:
            domain_name = site
        domains.append(domain_name)
    return domains

In [22]:
def tokenize_subdom(string):
    """
    Extracts domains and subdomains from a website list string.
    
    Args:
    string (str): string to be tokenized
    Out:
    subdomains + domains + dom/subdom combinations(list): list of tokens
    """
    sites = string.split(' ')
    subdomains = []
    domains = []
    dom_subdom = []
    for site in sites:
        try:
            domains.append(site.split('.')[-2])
            dom_subdom.append(site.split('.')[-2])
            subdomains.append(site.split('.')[-3])
            dom_subdom.append(site.split('.')[-3])
        except:
            pass
        
    return subdomains + domains + dom_subdom

In [23]:
def domain_stats(string):
    """
    Calcs the number of subdomains in a session,
    and the fraction of unique subdomains in a session,
    also the number of domains and fraction of unique
    domains
    Args:
    string (str): sequence of websites encoded in a string
    """
    sites = string.split(' ')
    subdomains = []
    domains = []
    for site in sites:
        try:
            domains.append(site.split('.')[-2])
            subdomains.append(site.split('.')[-3])
        except:
            pass
    #print(domains)
    #print(subdomains)
    dom_cnt = len(domains)
    subdom_cnt = len(subdomains)
    #print((dom_cnt, subdom_cnt))
    if dom_cnt:
        dom_unique = len(set(domains)) / dom_cnt
    else:
        dom_unique = 1
        
    if subdom_cnt:
        subdom_unique = len(set(subdomains)) / subdom_cnt
    else:
        subdom_unique = 1
    return ((dom_cnt, dom_unique),(subdom_cnt, subdom_unique))

In [24]:
# Constants
TEST_SIZE = 0.1
RANDOM_STATE = 42
N_FOLDS = 7
SKIP_FOLDS = 2

# Part 1. Basic TF-IDF. No feature engineering or hyperparameter optimization.

## Data load - basic

In [25]:
# Read the training and test data sets, change paths if needed
PATH_TO_DATA = Path('data/')

times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]
train_df = pd.read_csv(PATH_TO_DATA / 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(PATH_TO_DATA / 'test_sessions.csv',
                      index_col='session_id', parse_dates=times)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

In [26]:
# Replace site ids with site names
site_dic = pickle.load(open(PATH_TO_DATA / 'site_dic.pkl','rb'))

# Make swap ids with values: make site ids dict keys
site_dic = {val: key for val, key in zip(site_dic.values(), site_dic.keys())}

# Replace site ids with domain names for both train and test
train_df[sites] = train_df[sites].applymap(lambda x: site_dic[x] if x in site_dic else x)
test_df[sites] = test_df[sites].applymap(lambda x: site_dic[x] if x in site_dic else x)

## Data preprocessing - basic

In [27]:
# Split off target
y = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

In [28]:
# Index to split the training and test data sets
idx_split = train_df.shape[0]

In [29]:
full_sites_strings = full_df[sites].apply(form_sentences, axis=1)

## Build basic pipeline

In [30]:
tokenize = tokenize_basic

In [31]:
basic_pipeline = Pipeline([
        ('site_bag',Pipeline([
            ('vect',CountVectorizer(tokenizer=tokenize,max_features=50000, ngram_range=(1,3))),
            ('tfidf',TfidfTransformer())
        ])),
    ('clf',LogisticRegression(C=1, 
                              solver='liblinear',random_state=RANDOM_STATE,class_weight=None))
])

## ROC AUC score for the basic pipeline

In [36]:
all_train_data = full_sites_strings.iloc[:idx_split]

In [37]:
ts_splitter = TimeSeriesSplit(n_splits=N_FOLDS)
cv_res = cross_val_score(estimator=basic_pipeline,X=all_train_data,y=y,cv=ts_splitter,n_jobs=7,scoring='roc_auc')
cv_res

array([0.70242979, 0.79771246, 0.79432459, 0.88165145, 0.88242881,
       0.88452159, 0.92673304])

In [39]:
print('Basic model cross-validation ROC AUC score is {}'.format(np.mean(cv_res[SKIP_FOLDS:])))

Basic model cross-validation ROC AUC score is 0.8739318976276982


# Part 2. Basic TF-IDF + feature engineering and data cleaning. No hyperparameter optimization.

## Data load - basic

In [40]:
# Read the training and test data sets, change paths if needed
PATH_TO_DATA = Path('data/')

times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]
train_df = pd.read_csv(PATH_TO_DATA / 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(PATH_TO_DATA / 'test_sessions.csv',
                      index_col='session_id', parse_dates=times)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

In [41]:
# Replace site ids with site names
site_dic = pickle.load(open(PATH_TO_DATA / 'site_dic.pkl','rb'))

# Make swap ids with values: make site ids dict keys
site_dic = {val: key for val, key in zip(site_dic.values(), site_dic.keys())}

# Replace site ids with domain names for both train and test
train_df[sites] = train_df[sites].applymap(lambda x: site_dic[x] if x in site_dic else x)
test_df[sites] = test_df[sites].applymap(lambda x: site_dic[x] if x in site_dic else x)

## Data load - remove abnormal data

In [42]:
# Remove everything before 12th of November
# This is because before this date data comes only on the 12th day of month
# and it decreases ROC AUC
train_df = train_df[train_df['time1']> datetime(2013,11,13)]

In [43]:
# Remove months where there were no Alice attacks from the train dataset: 
# Alice just didn't show up in these months, so the data is noise

In [44]:
train_df['start_yearmonth'] = train_df[times].min(axis=1).apply(lambda x: x.year*100 + x.month)
alice_by_month = train_df.groupby(by=['start_yearmonth'])['target'].sum()
months_to_drop = alice_by_month[alice_by_month==0].index.tolist()
train_df = train_df[~train_df['start_yearmonth'].isin(months_to_drop)]
train_df = train_df.drop('start_yearmonth',axis=1)

In [45]:
# Split off target
y = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

## Data preprocessing - basic

In [46]:
# Split off target
y = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

In [47]:
# Index to split the training and test data sets
idx_split = train_df.shape[0]

In [48]:
full_sites_strings = full_df[sites].apply(form_sentences, axis=1)

## Feature engineering
For reasoning that led to this set of features, see `visual_exploration.ipynb`. Among the ideas listed there, I picked ones that gave higher scores on cross-validation and Public Leaderboard.

In [49]:
extra_feats = pd.DataFrame(index=full_df.index)

extra_feats['session_length'] = (full_df[times].max(axis=1) - full_df[times].min(axis=1))/np.timedelta64(1, 's')
extra_feats['start_hour'] = full_df[times].min(axis=1).apply(lambda x: (x.hour))
extra_feats['start_minute'] = full_df[times].min(axis=1).apply(lambda x: (x.minute))
extra_feats['session_std'] = ((full_df[times]-datetime(1970,1,1))/np.timedelta64(1, 's')).std(axis=1)
# Additional factors for logit
extra_feats['weekday'] = full_df[times].min(axis=1).apply(lambda x: x.weekday())
extra_feats['yearmonth'] = full_df[times].min(axis=1).apply(lambda x: x.year*12+x.month)

In [50]:
logit_extra_all = extra_feats.copy().fillna(0)

In [51]:
# Add features for hour
# Based on visual analysis above
logit_extra_all['low_hours'] = logit_extra_all['start_hour'].isin([9,11,14,15]).astype(int)
logit_extra_all['midday'] = logit_extra_all['start_hour'].isin([12,13]).astype(int)
logit_extra_all['peak'] = logit_extra_all['start_hour'].isin([16,17]).astype(int)
logit_extra_all['evening'] = logit_extra_all['start_hour'].isin([18]).astype(int)
logit_extra_all['dead_hours'] = logit_extra_all['start_hour'].isin([0,1,2,3,4,5,6,7,8,10,19,20,21,24]).astype(int)

In [52]:
logit_extra_all['weekday_0_1'] = logit_extra_all['weekday'].isin([0,1]).astype(int)
logit_extra_all['weekday_3_4'] = logit_extra_all['weekday'].isin([3,4]).astype(int)
logit_extra_all['weekday_other'] = logit_extra_all['weekday'].isin([2,5,6]).astype(int)

In [53]:
# Calc features based on strings
stats = full_sites_strings.apply(domain_stats)

logit_extra_all['dom_cnt'] = stats.apply(lambda x: x[0][0])
logit_extra_all['dom_unique'] = stats.apply(lambda x: x[0][1])
logit_extra_all['subdom_cnt'] = stats.apply(lambda x: x[1][0])
logit_extra_all['subdom_unique'] = stats.apply(lambda x: x[1][1])

logit_extra_all['su_1'] = logit_extra_all['subdom_unique'].apply(lambda x: 1 if x==1 else 0)
logit_extra_all['su_05_1'] = logit_extra_all['subdom_unique'].apply(lambda x: 1 if x<1 and x > 0.5 else 0)
logit_extra_all['su_05'] = logit_extra_all['subdom_unique'].apply(lambda x: 1 if x==0.5 else 0)
logit_extra_all['su_02_05'] = logit_extra_all['subdom_unique'].apply(lambda x: 1 if x>=0.2 and x<0.5 else 0)
logit_extra_all['su_0'] = logit_extra_all['subdom_unique'].apply(lambda x: 1 if x<0.2 else 0)

#logit_extra_all['ud_08_1'] = logit_extra_all['dom_unique'].apply(lambda x: 1 if x>0.8 else 0)`

In [54]:
# Let's try some combinations
# Tried a bunch. What remains are only ones who give a consistent improvement
# over OOT cross-validation.
logit_extra_all['extra_busy'] = (logit_extra_all['weekday_0_1'] | logit_extra_all['weekday_3_4']) & \
                                (logit_extra_all['midday'] | logit_extra_all['peak'])

In [55]:
logit_extra_all = logit_extra_all.drop(['start_hour','start_minute','weekday',
                                        'dom_cnt',
                                        'dom_unique',
                                        'subdom_cnt','subdom_unique',
                                       ], axis=1)

## Merge additional features with TF-IDF

In [56]:
all_train_data = pd.concat([full_sites_strings.iloc[:idx_split],
                            logit_extra_all.iloc[:idx_split,:]],axis=1)
all_test_data = pd.concat([full_sites_strings.iloc[idx_split:],
                            logit_extra_all.iloc[idx_split:,:]],axis=1)

## Build a pipeline
__Note:__ We'll scale all the data except for TF-IDF output. TF-IDF transformer output must not be scaled as the different average values of different features are intentional. Details are in the blog post linked in the README to this project.

In [57]:
tokenize = tokenize_basic

In [58]:
main_pipeline = Pipeline([
    ('col_transform',ColumnTransformer([
        ('site_bag',Pipeline([
            ('vect',CountVectorizer(tokenizer=tokenize,max_features=50000, ngram_range=(1,3), stop_words=['www'])),
            ('tfidf',TfidfTransformer())
        ]),0),
        ('extra',StandardScaler(),slice(1,1+logit_extra_all.shape[1]))
    ],n_jobs=2)),
    ('clf',LogisticRegression(C=1, 
                              solver='liblinear',random_state=RANDOM_STATE,class_weight=None))
])

## ROC AUC score after feature engineering and data cleaning

In [60]:
ts_splitter = TimeSeriesSplit(n_splits=N_FOLDS)
cv_res = cross_val_score(estimator=main_pipeline,X=all_train_data,y=y,cv=ts_splitter,n_jobs=7,scoring='roc_auc')
cv_res

array([0.8891695 , 0.9647462 , 0.94316376, 0.96375194, 0.91808965,
       0.97105816, 0.98533805])

In [61]:
print('Feature engineering adn data cleaning model cross-validation ROC AUC score is {}'.format(np.mean(cv_res[SKIP_FOLDS:])))

Feature engineering adn data cleaning model cross-validation ROC AUC score is 0.9562803120927033


# Part 2. Basic TF-IDF + feature engineering and data cleaning. No hyperparameter optimization.

We don't need to reengineer the data. It's enough to just run hypeoptimization. All we need to do is set up a function that will calculate the loss of a hyperoptimization iteration and the hyperopt module will take care of the rest.

The loss function will be `-roc_auc_score`.

In [66]:
# This is hyperoptimization
def objective(space):
    """
    Takes hyperparameters and outputs negative
    cross-validation ROC AUC score
    
    space (dict): dictionary of hyperparameters
    Out:
    (loss,status): loss is negative ROC AUC score
    """
    #print(space)
    main_pipeline = Pipeline([
        ('col_transform',ColumnTransformer([
            ('site_bag',Pipeline([
                ('vect',CountVectorizer(tokenizer=space['vect__tokenizer'],max_features=space['vect__max_features']
                                        , ngram_range=(1,space['vect__ngram_max']), stop_words=space['vect__stop_words'])),
                ('tfidf',TfidfTransformer())
            ]),0),
            ('extra',StandardScaler(),slice(1,1+logit_extra_all.shape[1]))
        ],n_jobs=2)),
        ('clf',LogisticRegression(C=space['clf__C'], 
                                  solver=space['clf__solver'],random_state=RANDOM_STATE,class_weight=None))
    ])
    
    ts_splitter = TimeSeriesSplit(n_splits=N_FOLDS)
    scores = cross_val_score(estimator=main_pipeline,X=all_train_data,y=y,cv=ts_splitter,n_jobs=7,scoring='roc_auc')
    loss = -np.mean(scores[SKIP_FOLDS:])
    return {'loss': loss, 'status': STATUS_OK}
    

In [67]:
space = {
    'vect__tokenizer': tokenize,
    'vect__max_features': hp.uniformint('vect__max_features',10000,100000),
    'vect__ngram_max': hp.uniformint('vect__ngram_max',1,10),
    'vect__stop_words': ['www'],
    'clf__C': hp.lognormal('clf__C',np.log(4.5),1),
    'clf__solver': 'liblinear'
}

In [68]:
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=50)

100%|██████████| 50/50 [45:28<00:00, 61.35s/it, best loss: -0.9662373627447648]


In [70]:
best

{'clf__C': 23.27535662532287,
 'vect__max_features': 46650.0,
 'vect__ngram_max': 1.0}

Save the result so that we won't have to rerun the entire thing.

In [71]:
best = {'clf__C': 23.27535662532287,
 'vect__max_features': 46650.0,
 'vect__ngram_max': 1.0}

# Final evaluation
We shouldn't forget to retrain the model on the entire dataset before submitting

In [72]:
settings = best
settings['vect__max_features'] = int(settings['vect__max_features'])
settings['vect__tokenizer'] = tokenize
settings['clf__solver'] = 'liblinear'

In [74]:
main_pipeline = Pipeline([
    ('col_transform',ColumnTransformer([
        ('site_bag',Pipeline([
            ('vect',CountVectorizer(tokenizer=settings['vect__tokenizer'],max_features=settings['vect__max_features']
                                    , ngram_range=(1,settings['vect__ngram_max']))),
            ('tfidf',TfidfTransformer())
        ]),0),
        ('extra',StandardScaler(),slice(1,1+logit_extra_all.shape[1]))
    ],n_jobs=2)),
    ('clf',LogisticRegression(C=settings['clf__C'], 
                              solver=settings['clf__solver'],random_state=RANDOM_STATE,class_weight=None))
])

In [75]:
main_pipeline.fit(all_train_data,y)

Pipeline(memory=None,
         steps=[('col_transform',
                 ColumnTransformer(n_jobs=2, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('site_bag',
                                                  Pipeline(memory=None,
                                                           steps=[('vect',
                                                                   CountVectorizer(analyzer='word',
                                                                                   binary=False,
                                                                                   decode_error='strict',
                                                                                   dtype=<class 'numpy.int64'>,
                                                                                   encoding='utf-8',
                                      

In [76]:
preds = main_pipeline.predict_proba(all_test_data)[:,1]

In [77]:

write_to_submission_file(preds, './results/final.csv')